In [ ]:
!pip install tensorflow_gpu

In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7



In [ ]:
import tensorflow as tf
print(tf.__version__)

2.6.0


In [ ]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.resnet152V2 import ResNet152V2
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [ ]:
import os

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/data/train')
valid_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/data/test')

In [ ]:
import tensorflow
resnet152V2 =tensorflow.keras.applications.ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


234561536/234545216 [==============================] - 2s 0us/step


In [ ]:
for layer in resnet152V2.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(resnet152V2.output)

In [ ]:
prediction = Dense (4, activation='softmax')(x)

# create a model object
model = Model(inputs=resnet152V2.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1951 images belonging to 4 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 18 images belonging to 5 classes.


In [ ]:
r = model.fit(
  training_set,
  steps_per_epoch=8,  
      epochs=20,
      verbose=1
)

Epoch 1/20
8/8 [==============================] - 101s 8s/step - loss: 3.1688 - accuracy: 0.5273
Epoch 2/20
8/8 [==============================] - 63s 8s/step - loss: 1.2362 - accuracy: 0.8477
Epoch 3/20
8/8 [==============================] - 48s 6s/step - loss: 1.0681 - accuracy: 0.8711
Epoch 4/20
8/8 [==============================] - 47s 6s/step - loss: 0.5729 - accuracy: 0.8711
Epoch 5/20
8/8 [==============================] - 34s 4s/step - loss: 0.7078 - accuracy: 0.9102
Epoch 6/20
8/8 [==============================] - 32s 4s/step - loss: 0.5982 - accuracy: 0.9102
Epoch 7/20
8/8 [==============================] - 29s 4s/step - loss: 0.8388 - accuracy: 0.8945
Epoch 8/20
8/8 [==============================] - 24s 3s/step - loss: 0.4626 - accuracy: 0.9294
Epoch 9/20
8/8 [==============================] - 24s 3s/step - loss: 1.0448 - accuracy: 0.8867
Epoch 10/20
8/8 [==============================] - 20s 2s/step - loss: 0.4102 - accuracy: 0.9453
Epoch 11/20
8/8 [=====================

In [ ]:
from tensorflow.keras.models import load_model

model.save('model_resnet152V2.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
from google.colab import files
files.download("model_resnet152V2.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')